<a href="https://colab.research.google.com/github/ilyaShevchuk/PyData/blob/main/laba_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа № 2**




In [ ]:
pip install pymorphy2

     |████████████████████████████████| 61kB 2.1MB/s 
     |████████████████████████████████| 8.2MB 5.0MB/s 


In [ ]:
from pandas import read_csv
import pymorphy2
import re
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.decomposition import PCA

In [ ]:
train = read_csv('https://static.tcsbank.ru/documents/olymp/passport_training_set.csv',';', index_col='id' ,encoding='cp1251')
train.shape

(96750, 3)

In [ ]:
train

,passport_div_code,passport_issuer_name,passport_issue_month/year
id,,,
1,422008,беловским управление внутренних дел кемеровско...,11M2001
2,500112,территориальный пункт в город орехово-зуево о...,03M2009
3,642001,волжским республика отдел внутренних дел город...,04M2002
4,162004,управление внутренних дел московского республ...,12M2002
5,80001,отделом отдел федеральной миграционной службы ...,08M2009
...,...,...,...
96750,262013,отделом внутренних дел город кисловодска ставр...,10M2001
96751,640004,отделом управление федеральной миграционной сл...,02M2008
96752,692001,заволжским отделом внутренних дел города твери,03M2006


In [ ]:
example_code = train.passport_div_code[train.passport_div_code.duplicated()].values[0]
for i in train.passport_issuer_name[train.passport_div_code == example_code].drop_duplicates():
    print (i)

ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖ. Р-Е
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО Р. КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСП КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ Р-НЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОУФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
УФМС РОССИИ ПО РК В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ МЕДВЕЖЬЕГОРСКОМ Р-ОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РК В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КОРЕЛИЯ В МЕДВЕЖИГОРСКОМ РАЙОНЕ
УФМС РОССИИ ПО Р. КАРЕЛИЯ МЕДВЕЖЬЕГОРСКОГО Р-НА
ОТДЕЛОМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ
УФМС РЕСПУБЛИКИ КАРЕЛИИ МЕДВЕЖЬЕГОРСКОГО Р-ОН
МЕДВЕЖЬЕГОРСКИМ ОВД


In [ ]:
train.passport_issuer_name = train.passport_issuer_name.str.lower()
train[train.passport_div_code == example_code].head(15)

,passport_div_code,passport_issuer_name,passport_issue_month/year
id,,,
19,100010,отделением уфмс россии по республике карелия в...,04M2008
22,100010,отделением уфмс россии по р. карелия в медвежь...,10M2009
5642,100010,отделением уфмс россии по респ карелия в медве...,08M2008
6668,100010,отделением уфмс россии по республике карелия в...,08M2011
8732,100010,отделением уфмс россии по республике карелия в...,08M2012
15637,100010,оуфмс россии по республике карелия в медвежьег...,08M2010
16749,100010,уфмс россии по рк в медвежьегорском районе,09M2007
17829,100010,оуфмс россии по республике карелия в медвежьег...,10M2008
25791,100010,отделением уфмс россии по республике карелия м...,06M2010


In [ ]:
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'р-(а|й|о|н|е)*',u'район')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' г( |\.|(ор(\.| )))', u' город ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' р(|\.есп )', u' республика ')


In [ ]:
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'адм[а-я]*\.',u'административный')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' ао ',u' административный округ ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'окр(\.|уга|)',u'округ')

In [ ]:
sokr = {u'нао': u'ненецкий автономный округ',
u'хмао': u'ханты-мансийский автономный округ',
u'чао': u'чукотский автономный округ',
u'янао': u'ямало-ненецкий автономный округ',
u'вао': u'восточный административный округ',
u'цао': u'центральный административный округ',
u'зао': u'западный административный округ',
u'cао': u'северный административный округ',
u'свао': u'северо-восточный округ',
u'сзао': u'северо-западный округ',
u'оуфмс': u'отдел управление федеральной миграционной службы',
u'офмс': u'отдел федеральной миграционной службы',
u'уфмс': u'управление федеральной миграционной службы',
u'увд': u'управление внутренних дел',
u'ровд': u'районный отдел внутренних дел',
u'говд': u'городской отдел внутренних дел',
u'рувд': u'районное управление внутренних дел',
u'овд': u'отдел внутренних дел',
u'оувд': u'отдел управления внутренних дел',
u'мро': u'межрайонный отдел',
u'юао':u'южный', 
u'юзао':u'юго-западный', 
u'ювао':u'юго-восточный', 
u'пс':u'паспортный стол', 
u'тп':u'территориальный пункт'}

In [ ]:
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' - ?', u'-')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'[^а-я -]','')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'- ',' ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'  *',' ')
print(train.keys)

<bound method NDFrame.keys of        passport_div_code  ... passport_issue_month/year
id                        ...                          
1                 422008  ...                   11M2001
2                 500112  ...                   03M2009
3                 642001  ...                   04M2002
4                 162004  ...                   12M2002
5                  80001  ...                   08M2009
...                  ...  ...                       ...
96750             262013  ...                   10M2001
96751             640004  ...                   02M2008
96752             692001  ...                   03M2006
96753             422007  ...                   01M2004
96754             742015  ...                   05M2003

[96750 rows x 3 columns]>


In [ ]:
for i in sokr.keys():
    train.passport_issuer_name = train.passport_issuer_name.str.replace(u'( %s )|(^%s)|(%s$)' % (i,i,i), u' %s ' % (sokr[i]))
    
#удалим лишние пробелы в конце и начале строки
train.passport_issuer_name = train.passport_issuer_name.str.lstrip()
train.passport_issuer_name = train.passport_issuer_name.str.rstrip()

## Анализ данных

In [ ]:
def f_tokenizer(s):
    path="/usr/local/lib/python3.6/dist-packages/pymorphy2_dicts_ru/data"
    morph = pymorphy2.MorphAnalyzer(path=path, lang='ru')
    if isinstance(s, str):
        t = s.split(' ')
    else:
        t = s
    f = []
    for j in t:
        m = morph.parse(j.replace('.',''))
        if len(m) != 0:
            wrd = m[0]
            if wrd.tag.POS not in ('NUMR','PREP','CONJ','PRCL','INTJ'):
                f.append(wrd.normal_form)
    return f


In [ ]:
coder = HashingVectorizer(tokenizer=f_tokenizer, n_features=256)

In [ ]:
TrainNotDuble = train.iloc[1:1000].drop_duplicates() 

In [ ]:
trn = coder.fit_transform(TrainNotDuble.passport_issuer_name.tolist()).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
target = TrainNotDuble.passport_div_code.values

In [ ]:
pca = PCA(n_components = 15)
trn = pca.fit_transform(trn)

In [ ]:
model = RandomForestClassifier(n_estimators = 100, criterion='entropy')

TRNtrain, TRNtest, TARtrain, TARtest = train_test_split(trn, target, test_size=0.4)
model.fit(TRNtrain, TARtrain)
print ('accuracy_score: ', accuracy_score(TARtest, model.predict(TRNtest)))

accuracy_score:  0.40350877192982454
